In [12]:
import math as mh
import multiprocessing as mg
import multiprocessing.pool
# import pys2let as ps
import random
import string
import itertools
import os

import jax
jax.config.update("jax_enable_x64", True)
import s2fft
import healpy as hp
import numpy as np
import s2wav
import s2wav
import matplotlib.pyplot as plt
%matplotlib inline 
import skyclean
from skyclean import CMB_data

In [10]:
# In Pixel space

stored_wavelet_coeffs_pix = [np.load(f"../convolution/wavelet_coefficient/wav_30_{i}.npy", allow_pickle=True) for i in range(12)]
stored_scaling_coeffs_pix = np.load("../convolution/scaling_coefficient/scal_30.npy")

In [46]:
# The extra dimension for the wavelet coefficients is for spin?
# Convert wavelet coefficient from MW pixel space to MW alm space
stored_wavelet_coeffs_pix[0].shape
stored_wavelet_coeffs_pix = stored_wavelet_coeffs_pix[:3]

stored_wavelet_coeffs_alm = []

for i in range(len(stored_wavelet_coeffs_pix)):
    print("Scale:",i,"original size", stored_wavelet_coeffs_pix[i].shape)
    stored_wavelet_coeffs_alm.append(s2fft.forward(stored_wavelet_coeffs_pix[i], L = stored_wavelet_coeffs_pix[i].shape[1]))
    print("Scale:",i,"alm size", stored_wavelet_coeffs_alm[i].shape)

print("finished converting wavelet coefficient from MW pixel to MW alm space")

stored_wavelet_coeffs_alm_doubled = []
for i in range(len(stored_wavelet_coeffs_alm)):
    print("Scale:",i,"original alm size", stored_wavelet_coeffs_alm[i].shape)
    padded_alm = np.zeros((stored_wavelet_coeffs_alm[i].shape[0]*2,stored_wavelet_coeffs_alm[i].shape[1]))
    # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
    padded_alm[:stored_wavelet_coeffs_alm[i].shape[0], :] = stored_wavelet_coeffs_alm[i]
    print("Scale:",i,"padded alm size", padded_alm.shape)
    stored_wavelet_coeffs_alm_doubled.append(padded_alm)
    # print("Scale:",i,"doubled alm size", stored_wavelet_coeffs_alm_doubled[i].shape)
    # print(padded_alm)

Scale: 0 original size (1, 4, 7)
Scale: 0 alm size (4, 7)
Scale: 1 original size (1, 4, 7)
Scale: 1 alm size (4, 7)
Scale: 2 original size (1, 8, 15)
Scale: 2 alm size (8, 15)
finished converting wavelet coefficient from MW pixel to MW alm space
Scale: 0 original alm size (4, 7)
Scale: 0 padded alm size (8, 7)
Scale: 0 doubled alm size (8, 7)
Scale: 1 original alm size (4, 7)
Scale: 1 padded alm size (8, 7)
Scale: 1 doubled alm size (8, 7)
Scale: 2 original alm size (8, 15)
Scale: 2 padded alm size (16, 15)
Scale: 2 doubled alm size (16, 15)


In [52]:
print(stored_wavelet_coeffs_alm_doubled[0].shape)


8

In [53]:
print(stored_wavelet_coeffs_alm_doubled[0].shape)
# print(np.multiply(stored_wavelet_coeffs_alm[0], stored_wavelet_coeffs_alm[0]).shape)
print(np.multiply(stored_wavelet_coeffs_alm_doubled[0], stored_wavelet_coeffs_alm_doubled[0]).shape)
# print(np.multiply(stored_wavelet_coeffs_alm_doubled[0], stored_wavelet_coeffs_alm_doubled[0]))

(8, 7)
(4, 7)
(8, 7)


In [54]:
a = np.array([[1,2],[3,4]])
b = np.array([[5,6],[7,8]])
display(a)
display(b)
display(np.multiply(a,b))

array([[1, 2],
       [3, 4]])

array([[5, 6],
       [7, 8]])

array([[ 5, 12],
       [21, 32]])

In [ ]:
# draft
# print(stored_wavelet_coeffs_pix[0])
# print(s2fft.inverse(stored_wavelet_coeffs_alm[0],L=stored_wavelet_coeffs_alm[0].shape[0]))
# print(stored_wavelet_coeffs_alm[0])

In [ ]:
# Shape of wavelet_coeffs:0 (1, 4, 7)
# Shape of wavelet_coeffs:1 (1, 4, 7)
# Shape of wavelet_coeffs:2 (1, 8, 15)
# Shape of wavelet_coeffs:3 (1, 16, 31)
# Shape of wavelet_coeffs:4 (1, 32, 63)
# Shape of wavelet_coeffs:5 (1, 64, 127)
# Shape of wavelet_coeffs:6 (1, 128, 255)
# Shape of wavelet_coeffs:7 (1, 256, 511)
# Shape of wavelet_coeffs:8 (1, 512, 1023)
# Shape of wavelet_coeffs:9 (1, 1024, 2047)
# Shape of wavelet_coeffs:10 (1, 2048, 4095)
# Shape of wavelet_coeffs:11 (1, 2048, 4095)

In [ ]:
def zeropad(i): #(alms[HPX],scale_lmax,smoothing_lmax) - COULD RE-WRITE FOR MW ALM'S???
    print "Zero-padding the alm's"

    #Pre-allocate array for enlarged alm's
    new_alms = np.zeros(((i[2]*(i[2]+1))/2.),dtype=complex)
    for em in xrange(i[1]):
        startindex = em*i[1] - .5*em*(em-1)
        new_startindex = em*i[2] - .5*em*(em-1)
        new_alms[new_startindex:(new_startindex+i[1]-em)] = i[0][startindex:(startindex+i[1]-em)]

    return new_alms

def doubleworker(i): #i = (j,n,map_index,scale_lmax,smoothing_lmax)
    print "Doubling l_max of input map", i[2]+1, "/", nmaps
    
    #Map loading within sub-process
    if i[0] >= 0: #Wavelet scales
        wav_fits = wav_fits_root[i[2]] + '_' + wavparam_code + str(i[0]) + '_n' + str(i[1]+1) + '.npy'
    else: #Scaling function
        wav_fits = scal_fits[i[2]]
    map = np.load(wav_fits,mmap_mode='r') #Map still only stored on disk

    alms = ps.map2alm_mw(map,i[3],spin) #alm's to l(j) - come out in MW order
    del map
    alms = zeropad((ps.lm2lm_hp(alms,i[3]),i[3],i[4])) #New alm's larger
    map = ps.alm2map_mw(ps.lm_hp2lm(alms,i[4]),i[4],spin) #Input alm's in MW order
    del alms
    
    #SAVE doubled map
    if i[0] >= 0: #Wavelet scales
        double_fits = wav_fits_root[i[2]] + '_' + wavparam_code + str(i[0]) + '_n' + str(i[1]+1) + '_double.npy'
    else: #Scaling function
        double_fits = scal_fits[i[2]][:-4] + '_double.npy'
    np.save(double_fits,map)
    del map
    
    return 0